In [3]:
#基本工具
import numpy as np
import pandas as pd
import time
import os #修改环境设置
import random
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import StandardScaler

from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import mutual_info_regression,mutual_info_classif
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.model_selection import cross_validate, KFold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import accuracy_score as ACC #当返回具体类别时，可以使用准确率
from sklearn.metrics import log_loss as logloss #当返回概率时，则必须使用交叉熵损失
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.linear_model import LogisticRegression as LogiR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC
# 自定义模块
from telcoFunc import *
# 导入特征衍生模块
import features_creation as fc
from features_creation import *
from tqdm import tqdm
import gc
#日常使用库与算法
import pandas as pd
import numpy as np
import sklearn
import matplotlib as mlp
import matplotlib.pyplot as plt
import time
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier as GBC
#导入优化算法
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss
#融合模型
from sklearn.ensemble import StackingClassifier

In [4]:
data = pd.read_csv("./6_1data_exp_shifted.csv",index_col=0)

data_test=pd.read_csv('./6_1data_exp_test_shifted.csv',index_col=0)

features = data.drop(columns='group').copy()
labels = data['group'].copy()

features_test = data_test.drop(columns='group').copy()
labels_test = data_test['group'].copy()

shift_gene=['ENSG00000164600.4', 'ENSG00000095970.12', 'ENSG00000171860.4',
       'ENSG00000135914.5', 'ENSG00000075275.12','ENSG00000131095.7']

features_keep_test=features_test.loc[:,shift_gene]

features_keep=features.loc[:,shift_gene]

scaler = StandardScaler()

scall=scaler.fit(features_keep)

ff_keep=scall.transform(features_keep)

ff_keep_test=scall.transform(features_keep_test)

ff_keep=pd.DataFrame(ff_keep)

ff_keep.columns=shift_gene

ff_keep_test=pd.DataFrame(ff_keep_test)

ff_keep_test.columns=shift_gene

In [5]:
ff_keep

,ENSG00000164600.4,ENSG00000095970.12,ENSG00000171860.4,ENSG00000135914.5,ENSG00000075275.12,ENSG00000131095.7
0,-0.330608,0.080920,1.048793,-0.929635,-0.177257,0.206507
1,-0.289473,0.170443,1.154591,-0.884262,-0.188482,0.337630
2,-0.275938,-0.229774,-0.153464,-0.884262,-0.175386,-0.627290
3,-0.294382,-0.171848,0.135077,-0.657401,-0.177257,-0.359111
4,-0.280317,-0.071794,-0.028430,-0.793518,-0.180999,-0.381304
...,...,...,...,...,...,...
1400,0.011742,-0.292967,0.962230,-0.611121,0.275493,3.680385
1401,-0.083532,-0.398287,-0.509333,-0.259033,-0.171644,-0.492424
1402,0.131697,0.307359,0.173549,-0.884262,-0.108035,1.217677
1403,0.545304,0.123049,0.231258,-0.263570,-0.108035,1.369595


In [6]:
Poly_train, Poly_test, colNames_train_new, colNames_test_new = Polynomial_Features(shift_gene, 
                                                                                   3, 
                                                                                   ff_keep, 
                                                                                   ff_keep_test, 
                                                                                   multi=False)

In [7]:
Poly_train

,ENSG00000164600.4**2*ENSG00000095970.12**0,ENSG00000164600.4**1*ENSG00000095970.12**1,ENSG00000164600.4**0*ENSG00000095970.12**2,ENSG00000164600.4**3*ENSG00000095970.12**0,ENSG00000164600.4**2*ENSG00000095970.12**1,ENSG00000164600.4**1*ENSG00000095970.12**2,ENSG00000164600.4**0*ENSG00000095970.12**3,ENSG00000164600.4**2*ENSG00000171860.4**0,ENSG00000164600.4**1*ENSG00000171860.4**1,ENSG00000164600.4**0*ENSG00000171860.4**2,...,ENSG00000135914.5**2*ENSG00000131095.7**1,ENSG00000135914.5**1*ENSG00000131095.7**2,ENSG00000135914.5**0*ENSG00000131095.7**3,ENSG00000075275.12**2*ENSG00000131095.7**0,ENSG00000075275.12**1*ENSG00000131095.7**1,ENSG00000075275.12**0*ENSG00000131095.7**2,ENSG00000075275.12**3*ENSG00000131095.7**0,ENSG00000075275.12**2*ENSG00000131095.7**1,ENSG00000075275.12**1*ENSG00000131095.7**2,ENSG00000075275.12**0*ENSG00000131095.7**3
0,0.109301,-0.026753,0.006548,-0.036136,0.008845,-0.002165,0.000530,0.109301,-0.346739,1.099966,...,0.178468,-0.039644,0.008807,0.031420,-0.036605,0.042645,-0.005569,0.006488,-0.007559,0.008807
1,0.083794,-0.049338,0.029051,-0.024256,0.014282,-0.008409,0.004951,0.083794,-0.334223,1.333081,...,0.264000,-0.100801,0.038488,0.035526,-0.063637,0.113994,-0.006696,0.011994,-0.021486,0.038488
2,0.076142,0.063403,0.052796,-0.021010,-0.017495,-0.014568,-0.012131,0.076142,0.042347,0.023551,...,-0.490491,-0.347951,-0.246834,0.030760,0.110018,0.393493,-0.005395,-0.019296,-0.069013,-0.246834
3,0.086661,0.050589,0.029532,-0.025511,-0.014893,-0.008694,-0.005075,0.086661,-0.039764,0.018246,...,-0.155199,-0.084779,-0.046311,0.031420,0.063655,0.128961,-0.005569,-0.011283,-0.022859,-0.046311
4,0.078577,0.020125,0.005154,-0.022027,-0.005641,-0.001445,-0.000370,0.078577,0.007969,0.000808,...,-0.240096,-0.115372,-0.055439,0.032761,0.069015,0.145393,-0.005930,-0.012492,-0.026316,-0.055439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,0.000138,-0.003440,0.085829,0.000002,-0.000040,0.001008,-0.025145,0.000138,0.011299,0.925887,...,1.374511,-8.277785,49.851693,0.075896,1.013919,13.545237,0.020909,0.279327,3.731614,49.851693
1401,0.006978,0.033270,0.158632,-0.000583,-0.002779,-0.013251,-0.063181,0.006978,0.042545,0.259420,...,-0.033041,-0.062811,-0.119404,0.029462,0.084522,0.242481,-0.005057,-0.014508,-0.041621,-0.119404
1402,0.017344,0.040478,0.094470,0.002284,0.005331,0.012441,0.029036,0.017344,0.022856,0.030119,...,0.952126,-1.311128,1.805494,0.011672,-0.131552,1.482737,-0.001261,0.014212,-0.160187,1.805494
1403,0.297356,0.067099,0.015141,0.162150,0.036589,0.008256,0.001863,0.297356,0.126106,0.053480,...,0.095145,-0.494402,2.569075,0.011672,-0.147964,1.875791,-0.001261,0.015985,-0.202651,2.569075


In [8]:
Poly_train.to_csv("105genes.csv")

In [9]:
MI = mutual_info_classif(Poly_train, labels, random_state=1412)


MI_threshold = MI.mean() * 0.1

In [12]:
max(MI)

0.0676748192403187

In [24]:
MI_Ploy_cols = []

for MIvalue, colname in zip(MI, colNames_train_new):
    if MIvalue > MI_threshold:
        MI_Ploy_cols.append(colname)

print(len(MI_Ploy_cols))    

93


In [25]:
MI_threshold

0.003010492395574965

In [26]:
Poly_train=Poly_train.loc[:,MI_Ploy_cols]
Poly_test=Poly_test.loc[:,MI_Ploy_cols]
Poly_train=pd.concat([Poly_train,ff_keep],axis=1)
Poly_test=pd.concat([Poly_test,ff_keep_test],axis=1)

In [27]:
Poly_train

,ENSG00000164600.4**1*ENSG00000095970.12**1,ENSG00000164600.4**0*ENSG00000095970.12**2,ENSG00000164600.4**3*ENSG00000095970.12**0,ENSG00000164600.4**2*ENSG00000095970.12**1,ENSG00000164600.4**1*ENSG00000095970.12**2,ENSG00000164600.4**0*ENSG00000095970.12**3,ENSG00000164600.4**1*ENSG00000171860.4**1,ENSG00000164600.4**0*ENSG00000171860.4**2,ENSG00000164600.4**3*ENSG00000171860.4**0,ENSG00000164600.4**2*ENSG00000171860.4**1,...,ENSG00000075275.12**3*ENSG00000131095.7**0,ENSG00000075275.12**2*ENSG00000131095.7**1,ENSG00000075275.12**1*ENSG00000131095.7**2,ENSG00000075275.12**0*ENSG00000131095.7**3,ENSG00000164600.4,ENSG00000095970.12,ENSG00000171860.4,ENSG00000135914.5,ENSG00000075275.12,ENSG00000131095.7
0,-0.026753,0.006548,-0.036136,0.008845,-0.002165,0.000530,-0.346739,1.099966,-0.036136,0.114635,...,-0.005569,0.006488,-0.007559,0.008807,-0.330608,0.080920,1.048793,-0.929635,-0.177257,0.206507
1,-0.049338,0.029051,-0.024256,0.014282,-0.008409,0.004951,-0.334223,1.333081,-0.024256,0.096748,...,-0.006696,0.011994,-0.021486,0.038488,-0.289473,0.170443,1.154591,-0.884262,-0.188482,0.337630
2,0.063403,0.052796,-0.021010,-0.017495,-0.014568,-0.012131,0.042347,0.023551,-0.021010,-0.011685,...,-0.005395,-0.019296,-0.069013,-0.246834,-0.275938,-0.229774,-0.153464,-0.884262,-0.175386,-0.627290
3,0.050589,0.029532,-0.025511,-0.014893,-0.008694,-0.005075,-0.039764,0.018246,-0.025511,0.011706,...,-0.005569,-0.011283,-0.022859,-0.046311,-0.294382,-0.171848,0.135077,-0.657401,-0.177257,-0.359111
4,0.020125,0.005154,-0.022027,-0.005641,-0.001445,-0.000370,0.007969,0.000808,-0.022027,-0.002234,...,-0.005930,-0.012492,-0.026316,-0.055439,-0.280317,-0.071794,-0.028430,-0.793518,-0.180999,-0.381304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,-0.003440,0.085829,0.000002,-0.000040,0.001008,-0.025145,0.011299,0.925887,0.000002,0.000133,...,0.020909,0.279327,3.731614,49.851693,0.011742,-0.292967,0.962230,-0.611121,0.275493,3.680385
1401,0.033270,0.158632,-0.000583,-0.002779,-0.013251,-0.063181,0.042545,0.259420,-0.000583,-0.003554,...,-0.005057,-0.014508,-0.041621,-0.119404,-0.083532,-0.398287,-0.509333,-0.259033,-0.171644,-0.492424
1402,0.040478,0.094470,0.002284,0.005331,0.012441,0.029036,0.022856,0.030119,0.002284,0.003010,...,-0.001261,0.014212,-0.160187,1.805494,0.131697,0.307359,0.173549,-0.884262,-0.108035,1.217677
1403,0.067099,0.015141,0.162150,0.036589,0.008256,0.001863,0.126106,0.053480,0.162150,0.068766,...,-0.001261,0.015985,-0.202651,2.569075,0.545304,0.123049,0.231258,-0.263570,-0.108035,1.369595


In [18]:
Poly_train.to_csv("99genes.csv")

In [29]:
 # basic model
clf1=XGBClassifier(n_estimators =91,learning_rate=0.24,booster='gbtree',gamma = 3.56,max_depth = 1,min_child_weight =2.1,subsample =0.59,colsample_bytree =0.12,colsample_bynode =0.88,reg_lambda =3.99,random_state=1412)
clf2=XGBClassifier(n_estimators =108,learning_rate=0.17,booster='gbtree',gamma = 5.98,max_depth = 1,min_child_weight =3.8,subsample =0.67,colsample_bytree =0.96,colsample_bynode =0.07,reg_lambda =2.69,random_state=420)
clf3= GBC(n_estimators = 81
              ,learning_rate = 0.15
              ,criterion = 'friedman_mse'
              ,loss = "exponential"
              ,max_depth = 1
              ,max_features = 30
              ,subsample = 0.72
              ,min_impurity_decrease = 0.98
              ,random_state=1412
              ,verbose=False)
clf4= GBC(n_estimators = 83
              ,learning_rate = 0.16
              ,criterion = 'friedman_mse'
              ,loss = "exponential"
              ,max_depth = 1
              ,max_features = 20
              ,subsample = 0.67
              ,min_impurity_decrease = 0.88
              ,random_state=420
              ,verbose=False)
clf5=RFC(criterion='entropy',max_depth=5,max_features=23,max_samples=0.64,min_impurity_decrease=0,n_estimators=51,random_state=1412)
clf6=SVC(C=0.13,kernel='linear')
clf7 = LogiR(max_iter = 3000, C=0.1, random_state=1412,n_jobs=8)
clf8 = KNNC(n_neighbors=20,n_jobs=8)


In [30]:
estimators = [("xgb1",clf1), ("xgb2", clf2)
              , ("GBDT1",clf3), ("GBDT2", clf4), ("RF",clf5) 
              , ("svm",clf6)
              , ("LR", clf7), ("knn", clf8)
             ]

In [35]:
def hyperopt_objective2(params):
    random.seed(int(params["seed"]))
    reg = LogiR(max_iter = 3000, C=params["C"], random_state=1412,n_jobs=12)
    clf = StackingClassifier(estimators=random.sample(estimators, int(params["n_e"]))
                         ,final_estimator=reg 
                         ,stack_method = "auto"
                         ,cv=5
                         ,n_jobs=12)

    validation_loss = cross_validate(clf,Poly_train,labels
                                     ,scoring="roc_auc"
                                     ,cv=KFold(n_splits=5,shuffle=True,random_state=1412)
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise')

    return  -np.mean(abs(validation_loss["test_score"]))

In [36]:
param_grid_simple2 = {
                      'C': hp.quniform("C",0.01,0.8,0.01)
                     ,'n_e':hp.quniform("n_e",2,8,1)
                     ,'seed':hp.quniform("seed",0,1000,1)
                    }

In [44]:
def param_hyperopt(max_evals=100):
    
    #保存迭代过程
    trials = Trials()
    
    #设置提前停止
    early_stop_fn = no_progress_loss(400)
    
    #定义代理模型
    params_best = fmin(hyperopt_objective3
                       , space = param_grid_simple3
                       , algo = tpe.suggest
                       , max_evals = max_evals
                       , verbose=True
                       , trials = trials
                       , early_stop_fn = early_stop_fn
                      )
    
    #打印最优参数，fmin会自动打印最佳分数
    print("\n","\n","best params: ", params_best,
          "\n")
    return params_best, trials

In [38]:
params_best, trials = param_hyperopt(10000)

  6%|▌         | 563/10000 [32:29<9:04:33,  3.46s/trial, best loss: -0.7705068588792384] 

 
 best params:  {'C': 0.6, 'n_e': 5.0, 'seed': 836.0} 



In [30]:
def hyperopt_objective(params):
    #random.seed(int(params["seed"]))
    reg = RFC(n_estimators = int(params["n_estimators"])
              ,max_depth = int(params["max_depth"])
              ,max_features = int(params["max_features"])
              ,min_impurity_decrease = params["min_impurity_decrease"]
              ,random_state=1412
              ,verbose=False
              ,n_jobs=-1)
    clf = StackingClassifier(estimators=estimators 
                         ,final_estimator=reg 
                         ,stack_method = "auto"
                         ,cv=5
                         ,n_jobs=8)

    validation_loss = cross_validate(clf,Poly_train,labels
                                     ,scoring="roc_auc"
                                     ,cv=KFold(n_splits=5,shuffle=True,random_state=1412)
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise')

    return  -np.mean(abs(validation_loss["test_score"]))

In [31]:
param_grid_simple = {'n_estimators': hp.quniform("n_estimators",80,120,1)
                     , 'max_depth': hp.quniform("max_depth",1,20,1)
                     , "max_features": hp.quniform("max_features",4,8,1)
                     , "min_impurity_decrease":hp.quniform("min_impurity_decrease",0,3,0.01)

                    }

In [32]:
def param_hyperopt(max_evals=100):
    
    #保存迭代过程
    trials = Trials()
    
    #设置提前停止
    early_stop_fn = no_progress_loss(400)
    
    #定义代理模型
    params_best = fmin(hyperopt_objective
                       , space = param_grid_simple
                       , algo = tpe.suggest
                       , max_evals = max_evals
                       , verbose=True
                       , trials = trials
                       , early_stop_fn = early_stop_fn
                      )
    
    #打印最优参数，fmin会自动打印最佳分数
    print("\n","\n","best params: ", params_best,
          "\n")
    return params_best, trials

In [34]:
params_best, trials = param_hyperopt(10000)

  4%|▍         | 443/10000 [44:57<16:09:57,  6.09s/trial, best loss: -0.7677451250292485]

 
 best params:  {'max_depth': 10.0, 'max_features': 5.0, 'min_impurity_decrease': 0.01, 'n_estimators': 99.0} 



In [31]:
#  last model
random.seed(int(836.0))
reg = LogiR(max_iter = 3000, C=0.6, random_state=1412,n_jobs=12)
clf = StackingClassifier(estimators=random.sample(estimators, int(5.0))  
                         ,final_estimator=reg 
                         ,stack_method = "auto"
                         ,cv=5
                         ,n_jobs=12)
validation_loss = cross_validate(clf,Poly_train,labels
                                     ,scoring="roc_auc"
                                     ,cv=KFold(n_splits=5,shuffle=True,random_state=1412)
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise')

In [32]:
np.mean(abs(validation_loss["test_score"]))

0.7703830449495559

In [44]:
clf.fit(Poly_train,labels)

StackingClassifier(cv=5,
                   estimators=[('LR',
                                LogisticRegression(C=0.1, max_iter=3000,
                                                   n_jobs=8,
                                                   random_state=1412)),
                               ('xgb1',
                                XGBClassifier(base_score=None, booster='gbtree',
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=0.88,
                                              colsample_bytree=0.12,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None, gamma=3.56,
                                              gpu_id=None, grow_policy...
                                              reg_alpha=None, reg_lambda=3.99, ...)),
                               ('RF',
                                RandomForestClassifier(criterion='entropy',
                                                       max_depth=5,
                                                       max_features=23,
                                                       max_samples=0.64,
                                                       min_impurity_decrease=0,
                                                       n_estimators=51,
                                                       random_state=1412)),
                               ('knn',
                                KNeighborsClassifier(n_jobs=8, n_neighbors=20)),
                               ('svm', SVC(C=0.13, kernel='linear'))],
                   final_estimator=LogisticRegression(C=0.6, max_iter=3000,
                                                      n_jobs=12,
                                                      random_state=1412),
                   n_jobs=12)

In [34]:
from sklearn.metrics import roc_curve as ROC
from sklearn.metrics import roc_auc_score, recall_score

In [45]:
area1 = roc_auc_score(labels_test,clf.predict_proba(Poly_test)[:,1])

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fbaab97e940>
Traceback (most recent call last):
  File "/home/omnisky/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/omnisky/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/omnisky/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/omnisky/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [46]:
area1

0.7405342080599813

In [37]:
clf.score(Poly_test,labels_test)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fbb8454b040>
Traceback (most recent call last):
  File "/home/omnisky/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/omnisky/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/omnisky/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/omnisky/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


0.7570977917981072